In [86]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from math import log, sqrt, pi, exp

#from pandas_datareader import data
from statsmodels.tsa import stattools
from statsmodels.tsa import arima_model
from statsmodels.graphics import gofplots
from statsmodels.regression import linear_model

from scipy import stats
import scipy.integrate
import scipy.special
# from arch.unitroot import PhillipsPerron
import yfinance as yf
from sklearn.linear_model import LinearRegression

import os
import matplotlib.pyplot as plt  # for plotting
from statsmodels.regression import linear_model

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)
import statistics
import scipy.stats as si
from scipy.stats import norm

Data Download

In [87]:
tickers = ["CSCO","BAC","GE"]
#downoad the prices and volumes for the previously read list of tickers
start_date = '2021-09-01'
end_date = '2022-07-05'
stock_data = yf.download(tickers, start=start_date, end=end_date)
print(stock_data)

[*********************100%***********************]  3 of 3 completed
            Adj Close                             Close             \
                  BAC       CSCO          GE        BAC       CSCO   
Date                                                                 
2021-08-31  40.901463  57.367805  105.016106  41.750000  59.020000   
2021-09-01  40.352840  57.387249  103.272659  41.189999  59.040001   
2021-09-02  40.392227  57.863529  105.862938  41.020000  59.529999   
2021-09-03  40.421772  57.756607  104.358574  41.049999  59.419998   
2021-09-07  40.687637  57.231728  102.904030  41.320000  58.880001   
...               ...        ...         ...        ...        ...   
2022-06-27  32.349998  43.379562   66.709999  32.349998  43.770000   
2022-06-28  32.259998  42.675900   65.879997  32.259998  43.060001   
2022-06-29  31.860001  42.497501   63.689999  31.860001  42.880001   
2022-06-30  31.129999  42.259644   63.669998  31.129999  42.639999   
2022-07-01  31.559999

Get Dividend yields of each stock

In [88]:
# Dividend Rates
csco = yf.Ticker("csco")
a_csco = csco.info['dividendRate']
a_csco = a_csco/100
print(a_csco)

ge = yf.Ticker("ge")
a_ge = ge.info['dividendRate']
a_ge = a_ge/100
print(a_ge)

bac = yf.Ticker("bac")
a_bac = bac.info['dividendRate']
a_bac = a_bac/100
print(a_bac)

0.0152
0.0032
0.0084


Get historical volatility of each stock


In [89]:
# Working on statistics
price = stock_data['Adj Close'].values
days = price.shape[0]
ret = price[1:]/price[:-1] - 1
#print(ret)

price_tab = np.array([])
ret_tab = np.array([])
for i in range(len(price)):
    price_tab = np.append(price_tab, price[i])
for i in range(len(ret)):
    ret_tab = np.append(ret_tab, ret[i])

price_tab = price_tab.reshape(days,3)
ret_tab = ret_tab.reshape(days-1,3)

df_price = pd.DataFrame(price_tab)
df_price.columns =['CSCO', 'BAC','GE']
df_ret = pd.DataFrame(ret_tab)
df_ret.columns=['CSCO', 'BAC','GE']
# Print prices
print(df_price)
#Print returns
print(df_ret)

hist_std_csco = np.std(df_ret["CSCO"])*np.sqrt(250)
hist_std_bac = np.std(df_ret["BAC"])*np.sqrt(250)
hist_std_ge = np.std(df_ret["GE"])*np.sqrt(250)

# Historical volatility
print(hist_std_csco)
print(hist_std_bac)
print(hist_std_ge)

          CSCO        BAC          GE
0    40.901463  57.367805  105.016106
1    40.352840  57.387249  103.272659
2    40.392227  57.863529  105.862938
3    40.421772  57.756607  104.358574
4    40.687637  57.231728  102.904030
..         ...        ...         ...
206  32.349998  43.379562   66.709999
207  32.259998  42.675900   65.879997
208  31.860001  42.497501   63.689999
209  31.129999  42.259644   63.669998
210  31.559999  42.219997   63.540001

[211 rows x 3 columns]
         CSCO       BAC        GE
0   -0.013413  0.000339 -0.016602
1    0.000976  0.008299  0.025082
2    0.000731 -0.001848 -0.014210
3    0.006577 -0.009088 -0.013938
4   -0.013069 -0.003397 -0.006874
..        ...       ...       ...
205  0.001238 -0.007483 -0.004328
206 -0.002782 -0.016221 -0.012442
207 -0.012399 -0.004180 -0.033242
208 -0.022913 -0.005597 -0.000314
209  0.013813 -0.000938 -0.002042

[210 rows x 3 columns]
0.30215323643408976
0.28364092454684375
0.33932396122449787



Yahoo Finance Quotes: Call + Put options


In [90]:
# Getting Call/Put options: Market Prices
opt_csco = csco.option_chain(date='2023-01-20')
options_cisco = opt_csco
print(options_cisco)

opt_bac = bac.option_chain(date='2023-01-20')
options_bac = opt_bac

opt_ge = ge.option_chain(date='2023-01-20')
options_ge = opt_ge

Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice    bid  \
0   CSCO230120C00020000 2022-07-01 17:39:59+00:00    20.0      22.10  22.55   
1   CSCO230120C00023000 2022-01-03 18:48:29+00:00    23.0      39.95  33.05   
2   CSCO230120C00025000 2022-07-01 18:16:32+00:00    25.0      17.15  17.70   
3   CSCO230120C00028000 2022-07-05 17:47:34+00:00    28.0      13.85  14.90   
4   CSCO230120C00030000 2022-07-05 13:30:14+00:00    30.0      12.34  13.10   
5   CSCO230120C00033000 2022-07-05 13:35:20+00:00    33.0       9.35  10.45   
6   CSCO230120C00035000 2022-07-05 16:45:07+00:00    35.0       7.80   8.75   
7   CSCO230120C00037000 2022-07-05 19:15:46+00:00    37.0       6.80   7.20   
8   CSCO230120C00040000 2022-07-06 16:30:14+00:00    40.0       5.10   5.10   
9   CSCO230120C00042000 2022-07-06 17:00:15+00:00    42.0       3.95   3.90   
10  CSCO230120C00045000 2022-07-06 14:59:02+00:00    45.0       2.43   2.48   
11  CSCO230120C00047000 2022-07-06 16:

In [91]:
# Get Market Call Prices - CSCO
strike_call_list_cisco = (options_cisco[0]['strike'])
market_price_call_cisco = (options_cisco[0]['lastPrice'])
market_implied_vol_call_cisco = (options_cisco[0]['impliedVolatility'])
df_market_call_csco = pd.DataFrame({'strike':strike_call_list_cisco ,
                   'lastPrice':market_price_call_cisco ,
                   'impliedVolatility': market_implied_vol_call_cisco})
#print(df_market_call)

# Get Market Put Prices - CSCO
strike_put_list_cisco = (options_cisco[1]['strike'])
market_price_put_cisco = (options_cisco[1]['lastPrice'])
market_implied_vol_put_cisco = (options_cisco[1]['impliedVolatility'])
df_market_put_cisco = pd.DataFrame({'strike':strike_put_list_cisco ,
                   'lastPrice':market_price_put_cisco ,
                   'impliedVolatility': market_implied_vol_put_cisco})

# Get Market Call Prices - GE
strike_call_list_ge = (options_ge[0]['strike'])
market_price_call_ge = (options_ge[0]['lastPrice'])
market_implied_vol_call_ge = (options_ge[0]['impliedVolatility'])
df_market_call_ge = pd.DataFrame({'strike':strike_call_list_ge ,
                   'lastPrice':market_price_call_ge ,
                   'impliedVolatility': market_implied_vol_call_ge})
#print(df_market_call)

# Get Market Put Prices - GE
strike_put_list_ge = (options_ge[1]['strike'])
market_price_put_ge = (options_ge[1]['lastPrice'])
market_implied_vol_put_ge = (options_ge[1]['impliedVolatility'])
df_market_put_ge = pd.DataFrame({'strike':strike_put_list_ge ,
                   'lastPrice':market_price_put_ge ,
                   'impliedVolatility': market_implied_vol_put_ge})

#print(df_market_call_ge)

Parameters: Black-Scholes

In [92]:
S_0_CSCO = 42.49            # stock price: CSCO
S_0_GE = 61.19              # stock price: GE
r = 1.85/100                # interest rate

sigma_CSCO = hist_std_csco  # historical (std) volatility CSCO
sigma_GE = hist_std_ge      # historical (std) volatility GE

K_CSCO = 55                 # specific strike CSCO
K_GE = 35                   # specific strike GE

T = 143/250                 # 30 weeks: 150 trading days
N = 10000                   # number of trajectories

# Getting the quoted implied volatility: CSCO
market_call_CSCO = df_market_call_csco.loc[df_market_call_csco['strike']==K_CSCO]
implied_vol_CSCO_Call = (np.array(market_call_CSCO["impliedVolatility"]))
implied_vol_CSCO_Call = implied_vol_CSCO_Call[0]

market_put_CSCO = df_market_put_cisco.loc[df_market_put_cisco['strike']==K_CSCO]
implied_vol_CSCO_Put = (np.array(market_put_CSCO["impliedVolatility"]))
implied_vol_CSCO_Put = implied_vol_CSCO_Put[0]

# Getting the quoted implied volatility: GE
market_call_ge = df_market_call_ge.loc[df_market_call_ge['strike']==K_GE]
implied_vol_ge_Call = (np.array(market_call_ge["impliedVolatility"]))
implied_vol_ge_Call = implied_vol_ge_Call[0]

market_put_ge = df_market_put_ge.loc[df_market_put_ge['strike']==K_GE]
implied_vol_ge_Put = (np.array(market_put_ge["impliedVolatility"]))
implied_vol_ge_Put = implied_vol_ge_Put[0]

Black-Scholes-Merton: Options Prices

In [93]:
def black_scholes_merton_call_div(S, K, T, r, a, sigma):

    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #a: dividend yield
    #sigma: volatility of underlying asset

    d1 = (np.log(S/K) + (r-a+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = (np.log(S/K) + (r-a-0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    call = (S*np.exp(-a*T) * si.norm.cdf(d1, 0.0, 1.0) - K*np.exp(-r*T) * si.norm.cdf(d2, 0.0, 1.0))
    return call


def black_scholes_put_div(S, K, T, r, a, sigma):

    d1_bs = (np.log(S/K) + (r-a+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2_bs = (np.log(S/K) + (r-a-0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    put = (K*np.exp(-r*T) * si.norm.cdf(-d2_bs, 0.0, 1.0) - S*np.exp(-a*T) * si.norm.cdf(-d1_bs, 0.0, 1.0))
    return put

Testing Black-Scholes functions: with historical volatility & implied volatility quoted by Yahoo Finance

In [94]:
# Black-Scholes CSCO results
call_bs_CSCO =  black_scholes_merton_call_div(S_0_CSCO, K_CSCO, T, r, a_csco, hist_std_csco)
put_bs_CSCO =  black_scholes_put_div(S_0_CSCO, K_CSCO, T, r, a_csco, hist_std_csco)

call_bs_CSCO_implied_vol =  black_scholes_merton_call_div(S_0_CSCO, K_CSCO, T, r, a_csco, implied_vol_CSCO_Call)
put_bs_CSCO_implied_vol =  black_scholes_put_div(S_0_CSCO, K_CSCO, T, r, a_csco, implied_vol_CSCO_Put)

# Black-Scholes GE results
call_bs_GE =  black_scholes_merton_call_div(S_0_GE, K_GE, T, r, a_ge, hist_std_ge)
put_bs_GE =  black_scholes_put_div(S_0_GE, K_GE, T, r, a_ge, hist_std_ge)

call_bs_GE_implied_vol =  black_scholes_merton_call_div(S_0_GE, K_GE, T, r, a_ge, implied_vol_ge_Call)
put_bs_GE_implied_vol =  black_scholes_put_div(S_0_GE, K_GE, T, r, a_ge, implied_vol_ge_Put)

Black-Scholes-Merton: Greeks


In [95]:
def d1(S,K,T,r,a,sigma):
    return(log(S/K)+(r-a+sigma**2/2.)*T)/(sigma*sqrt(T))

def d2(S,K,T,r,a,sigma):
    return d1(S,K,T,r,a,sigma)-sigma*sqrt(T)

############## CALL OPTION ##############

def call_delta(S,K,T,r,a,sigma):
    return exp(-a*T)*norm.cdf(d1(S,K,T,r,a,sigma))

def call_gamma(S,K,T,r,a,sigma):
    return exp(-a*T)*norm.pdf(d1(S,K,T,r,a,sigma))/(S*sigma*sqrt(T))

def call_vega(S,K,T,r,a,sigma):
    return 0.01*exp(-a*T)*(S*norm.pdf(d1(S,K,T,r,a,sigma))*sqrt(T))

############## PUT OPTION ##############

def put_delta(S,K,T,r,a,sigma):
    return -exp(-a*T)*norm.cdf(-d1(S,K,T,r,a,sigma))

def put_gamma(S,K,T,r,a,sigma):
    return exp(-a*T)*norm.pdf(d1(S,K,T,r,a,sigma))/(S*sigma*sqrt(T))

def put_vega(S,K,T,r,a,sigma):
    return 0.01*exp(-a*T)*(S*norm.pdf(d1(S,K,T,r,a,sigma))*sqrt(T))

Greeks Testing: with historical volatility

In [96]:
# GREEKS
print("USING HISTORICAL VOLATILITY")
print()
# CSCO:
delta_CSCO = call_delta(S_0_CSCO,K_CSCO,T,r,a_csco,hist_std_csco)
print("Delta CSCO: ", delta_CSCO)
gamma_CSCO = call_gamma(S_0_CSCO,K_CSCO,T,r,a_csco,hist_std_csco)
print("Gamma CSCO: ", gamma_CSCO)
vega_CSCO = call_vega(S_0_CSCO,K_CSCO,T,r,a_csco,hist_std_csco)
print("Vega CSCO: ", vega_CSCO)

# GE:
delta_GE = put_delta(S_0_GE,K_GE,T,r,a_ge,hist_std_ge)
print("Delta GE: ", delta_GE)
gamma_GE = put_gamma(S_0_GE,K_GE,T,r,a_ge,hist_std_ge)
print("Gamma GE: ", gamma_GE)
vega_GE = put_vega(S_0_GE,K_GE,T,r,a_ge,hist_std_ge)
print("Delta GE: ", vega_GE)

USING HISTORICAL VOLATILITY

Delta CSCO:  0.15566505302056396
Gamma CSCO:  0.024537362848814265
Vega CSCO:  0.07656400210787105
Delta GE:  -0.009579166147223457
Gamma GE:  0.0016332216580074335
Delta GE:  0.011884590068902684


Monte-Carlo Simulation


In [97]:
# Monte Carlo simulation: European Option

call_payoff_maturity_CSCO = []
put_payoff_maturity_CSCO = []
call_payoff_maturity_GE = []
put_payoff_maturity_GE = []


days_test = 150 # 1 week = 5 trading days
dt = 1/250

S_CSCO = np.zeros((days_test+1, N))
S_CSCO[0] = S_0_CSCO
S_GE = np.zeros((days_test+1, N))
S_GE[0] = S_0_GE

for t in range(1, days_test+1):
    Z_CSCO = np.random.normal(size=N)
    Z_GE = np.random.normal(size=N)
    S_CSCO[t] = S_CSCO[t-1]*np.exp((r-a_csco-0.5*sigma_CSCO**2)*dt+sigma_CSCO*np.sqrt(dt)*Z_CSCO)
    S_GE[t] = S_GE[t-1]*np.exp((r-a_ge-0.5*sigma_GE**2)*dt+sigma_GE*np.sqrt(dt)*Z_GE)

Valuation (European)

In [98]:
S_T_CSCO = S_CSCO[-1]
S_T_GE = S_GE[-1]
for i in range(len(S_T_CSCO)):
    call_payoff_maturity_CSCO.append(max(S_T_CSCO[i]-K_CSCO,0))
    put_payoff_maturity_CSCO.append(max(K_CSCO-S_T_CSCO[i],0))

    call_payoff_maturity_GE.append(max(S_T_GE[i]-K_GE,0))
    put_payoff_maturity_GE.append(max(K_GE-S_T_GE[i],0))

# Price of CSCO options: expiry on Jan 20, 2023
VMC_call_CSCO = np.mean(call_payoff_maturity_CSCO)
C_0_CSCO = VMC_call_CSCO*np.exp(-r*days_test/250)
print(C_0_CSCO)

VMC_put_CSCO = np.mean(put_payoff_maturity_CSCO)
P_0_CSCO = VMC_put_CSCO*np.exp(-r*days_test/250)
print(P_0_CSCO)

# Price of GE options: expiry on Jan 20, 2023
VMC_call_GE = np.mean(call_payoff_maturity_GE)
C_0_GE = VMC_call_GE*np.exp(-r*days_test/250)
print(C_0_GE)

VMC_put_GE = np.mean(put_payoff_maturity_GE)
P_0_GE = VMC_put_GE*np.exp(-r*days_test/250)
print(P_0_GE)

0.7827131100545617
12.966337770873968
26.498699446292747
0.07025862915695348


Recap Results: Market Prices, Black-Scholes formulas, Black-Scholes Monte-Carlo

In [99]:
# Market Prices
market_call_CSCO = df_market_call_csco.loc[df_market_call_csco['strike']==K_CSCO]
market_put_CSCO = df_market_put_cisco.loc[df_market_put_cisco['strike']==K_CSCO]

print("CSCO Call prices:")
print("CSCO Market call: ", market_call_CSCO)
print("CSCO Black-Scholes call with market implied volatility: ", call_bs_CSCO_implied_vol)
print("CSCO Black-Scholes call with historical volatility: ", call_bs_CSCO)
print("EU CSCO call with MC estimation: ", C_0_CSCO)
# print()
# print("CSCO Put prices:")
# print("CSCO Market put: ", market_put_CSCO)
# print("CSCO Black-Scholes put with implied volatility: ", put_bs_CSCO_implied_vol)
# print("CSCO Black-Scholes put with historical volatility: ", put_bs_CSCO)
# print("EU CSCO put with MC estimation: ", P_0_CSCO)


CSCO Call prices:
CSCO Market call:      strike  lastPrice  impliedVolatility
14    55.0       0.34           0.255867
CSCO Black-Scholes call with market implied volatility:  0.3996044495171782
CSCO Black-Scholes call with historical volatility:  0.7170051395472683
EU CSCO call with MC estimation:  0.7827131100545617


In [100]:
market_call_ge = df_market_call_ge.loc[df_market_call_ge['strike']==K_GE]
market_put_ge = df_market_put_ge.loc[df_market_put_ge['strike']==K_GE]

# print("GE Call prices:")
# print("GE Market call: ", market_call_ge)
# print("GE Black-Scholes call with implied volatility: ", call_bs_GE_implied_vol)
# print("GE Black-Scholes call with historical volatility: ", call_bs_GE)
# print("EU GE call with MC estimation: ", C_0_GE)
# print()
print("GE Put prices:")
print("GE Market put: ", market_put_ge)
print("GE Black-Scholes put with market implied volatility: ", put_bs_GE_implied_vol)
print("GE Black-Scholes put with historical volatility: ", put_bs_GE)
print("EU GE put with MC estimation: ", P_0_GE)

GE Put prices:
GE Market put:      strike  lastPrice  impliedVolatility
12    35.0       0.65           0.536626
GE Black-Scholes put with market implied volatility:  0.6753334024949811
GE Black-Scholes put with historical volatility:  0.055163971655895705
EU GE put with MC estimation:  0.07025862915695348


In [101]:
# Build own implied volatility using Black-Scholes model

def call_implied_volatility(Price, S, K, T, r, a):
    implied_vol_bs = 0.001
    while implied_vol_bs < 1:
        Price_implied = S * exp(-a*T) * norm.cdf(d1(S, K, T, r, a, implied_vol_bs))-K*exp(-r*T) * \
            norm.cdf(d2(S, K, T, r, a, implied_vol_bs))
        if Price- Price_implied < 0.001:
            return implied_vol_bs
        implied_vol_bs += 0.001
    return "Not Found"

def put_implied_volatility(Price, S, K, T, r, a):
    implied_vol_bs = 0.001
    while implied_vol_bs < 1:
        Price_implied = K*exp(-r*T)-S*exp(-a*T)+black_scholes_merton_call_div(S, K, T, r, a, implied_vol_bs)
        if Price- Price_implied < 0.001:
            return implied_vol_bs
        implied_vol_bs += 0.001
    return "Not Found"



Using quotes, we get implied volatilities using Black-Scholes model. Not always a quote for Put option GE, K=10. Therefore, we take at the money put option. In BSM, it is supposed to be constant for all the strikes across the same maturity: it justifies why it can be done but it emphasizes this limitation of BSM and its implied volatility assumption.


In [109]:
# Implied volatility for Black-Scholes model
K_at_the_money_CSCO = 55
K_at_the_money_GE = 35

implied_black_scholes_CSCO = call_implied_volatility(0.34, S_0_CSCO, K_CSCO, T, r, a_csco)
bsm_call_CSCO_implied = black_scholes_merton_call_div(S_0_CSCO, K_CSCO, T, r, a_csco, implied_black_scholes_CSCO)
print("BSM: Implied volatility CSCO, K=55: ", implied_black_scholes_CSCO)
print("BSM: Price using implied volatility CSCO, K=55: ", bsm_call_CSCO_implied)

implied_black_scholes_GE = put_implied_volatility(0.65, S_0_GE, K_at_the_money_GE, T, r, a_ge)
bsm_put_GE_implied = black_scholes_put_div(S_0_GE, K_GE, T, r, a_ge, implied_black_scholes_GE)
print("BSM: At the money implied volatility GE: ", implied_black_scholes_GE)
print("BSM: At the money Price using implied volatility GE: ", bsm_put_GE_implied)

BSM: Implied volatility CSCO, K=55:  0.2460000000000002
BSM: Price using implied volatility CSCO, K=55:  0.3425553777108017
BSM: At the money implied volatility GE:  0.5320000000000004
BSM: At the money Price using implied volatility GE:  0.6519087627993945



Greeks with implied volatility estimated with Black-Scholes

In [103]:
# GREEKS
print("USING BLACK SCHOLES ESTIMATED IMPLIED VOLATILITY")
print()
# CSCO:
delta_CSCO = call_delta(S_0_CSCO,K_CSCO,T,r,a_csco,implied_black_scholes_CSCO)
print("Delta CSCO: ", delta_CSCO)
gamma_CSCO = call_gamma(S_0_CSCO,K_CSCO,T,r,a_csco,implied_black_scholes_CSCO)
print("Gamma CSCO: ", gamma_CSCO)
vega_CSCO = call_vega(S_0_CSCO,K_CSCO,T,r,a_csco,implied_black_scholes_CSCO)
print("Vega CSCO: ", vega_CSCO)

# GE:
delta_GE = put_delta(S_0_GE,K_GE,T,r,a_ge,implied_black_scholes_GE)
print("Delta GE: ", delta_GE)
gamma_GE = put_gamma(S_0_GE,K_GE,T,r,a_ge,implied_black_scholes_GE)
print("Gamma GE: ", gamma_GE)
vega_GE = put_vega(S_0_GE,K_GE,T,r,a_ge,implied_black_scholes_GE)
print("Vega GE: ", vega_GE)

USING BLACK SCHOLES ESTIMATED IMPLIED VOLATILITY

Delta CSCO:  0.08322521292124366
Gamma CSCO:  0.020588235233570312
Vega CSCO:  0.049113466975928395
Delta GE:  -0.05573636405648926
Gamma GE:  0.004477129841387157
Vega GE:  0.052038350717949


In [108]:
# GREEKS WITH PARAMETERS FIXED FOR BLACK-SCHOLES USE IN THE FUTURE

# CSCO:
call_def = black_scholes_merton_call_div(S_0_CSCO, K_CSCO, T, 1.68/100, a_csco, 0.254)
print(call_def)
delta_CSCO = call_delta(S_0_CSCO,K_CSCO,T,1.68/100,a_csco,0.254)
print("Delta CSCO: ", delta_CSCO)
gamma_CSCO = call_gamma(S_0_CSCO,K_CSCO,T,1.68/100,a_csco,0.254)
print("Gamma CSCO: ", gamma_CSCO)
vega_CSCO = call_vega(S_0_CSCO,K_CSCO,T,1.68/100,a_csco,0.254)
print("Vega CSCO: ", vega_CSCO)

# GE:
put_def = black_scholes_put_div(S_0_GE, K_GE, T, 1.68/100, a_ge, 0.539)
print(put_def)
delta_GE = put_delta(S_0_GE,K_GE,T,1.68/100,a_ge,0.539)
print("Delta GE: ", delta_GE)
gamma_GE = put_gamma(S_0_GE,K_GE,T,1.68/100,a_ge,0.539)
print("Gamma GE: ", gamma_GE)
vega_GE = put_vega(S_0_GE,K_GE,T,1.68/100,a_ge,0.539)
print("Vega GE: ", vega_GE)

0.38447358356102157
Delta CSCO:  0.10608910587058205
Gamma CSCO:  0.022389865828254186
Vega CSCO:  0.05872928298580451
0.6914617098556604
Delta GE:  -0.055266753676234966
Gamma GE:  0.004472081709059067
Vega GE:  0.05169196513869905
